In [10]:
import pydicom
import glob
import numpy as np
import datetime
import tifffile as tiff
import cv2 as cv
import plotly.express as px

In [11]:
def get_images_data(path: str) -> tuple[np.ndarray, list[str]]:
    """
    Carrega as imagens DICOM de um diretório e retorna um array NumPy contendo
    os dados de pixel das imagens e uma lista de nomes de arquivo.

    Args:
        path (str): Caminho para o diretório contendo os arquivos DICOM.

    Returns:
        Tuple[np.ndarray, List[str]]: Uma tupla contendo o array NumPy com os
        dados de pixel das imagens e uma lista de nomes de arquivo.

    Raises:
        FileNotFoundError: Se o caminho para o diretório especificado não existir.
    """
    files = glob.glob(path + '/*.dcm') # Obtendo uma lista de todos os arquivos DICOM no diretório
    #files.sort() # Organizando os arquivos em ordem ascendente
    image_data = []
    for i in range(len(files)): # Carregando as imagens DICOM
        ds = pydicom.dcmread(files[i])
        image_data.append(ds.pixel_array)
    return np.asarray(image_data), files

In [12]:
def create_DICOM(image_array: np.ndarray, series_description: str, output_path: str):
    """
    Cria um novo conjunto de dados DICOM a partir de uma matriz de imagem e uma descrição da série.

    Args:
        image_array (np.ndarray): Matriz de imagem (2D) representando a imagem a ser convertida para DICOM.
        series_description (str): Descrição da série DICOM.
        output_path (str): Caminho em que será salvo os arquivos.

    Returns:
        None
    """

    # Criar um novo conjunto de dados DICOM
    ds = pydicom.Dataset()

    # Definir tags DICOM para campos obrigatórios
    ds.SeriesDescription = series_description
    ds.Rows = image_array.shape[0]
    ds.Columns = image_array.shape[1]
    ds.PixelSpacing = [1.5625, 1.5625]
    ds.SliceThickness = 3.27
    ds.BitsAllocated = 8
    ds.BitsStored = 8
    ds.HighBit = 7
    ds.PixelRepresentation = 0
    ds.SamplesPerPixel = 1
    ds.PhotometricInterpretation = "MONOCHROME2"

    # Converter o array NumPy em bytes
    image_bytes = image_array.tobytes()

    # Atribuir a imagem em bytes no atributo PixelData
    ds.PixelData = image_bytes

    # Definir endianness e VR encoding
    ds.is_little_endian = True
    ds.is_implicit_VR = True

    # Criar o conjunto de dados FMI
    fmi = pydicom.dataset.FileMetaDataset()
    fmi.MediaStorageSOPClassUID = '1.2.840.10008.5.1.4.1.1.7'  # CT Image Storage
    fmi.MediaStorageSOPInstanceUID = '1.2.3.4.5.6.7.8.9.0'  # UID único
    fmi.TransferSyntaxUID = '1.2.840.10008.1.2.1'  # Explicit VR Little Endian
    
    # Atribuir o conjunto de dados FMI no conjunto de dados principal
    ds.file_meta = fmi

    # Definir a data e horário de criação
    dt = datetime.datetime.now()
    ds.ContentDate = dt.strftime('%Y%m%d')
    timeStr = dt.strftime('%H%M%S.%f')  # formato longo com micro segundos
    ds.ContentTime = timeStr

    # Salvar o conjunto de dados DICOM em um arquivo
    filename = series_description + '.dcm'
    ds.save_as(output_path + filename)

In [13]:
# Ler as imagens de referência
path = 'G:/My Drive/Projetos Python/Image Processing/Analise_de_Uniformidade_e_Ruido/dados/Cilindrico J-ADNI HCPA/CILINDRICO J-ADNI/VPFX-S2i32s3mm'
ref_images, file_names = get_images_data(path)

In [14]:
# Verificar voxel size das imagens de referência
ds = pydicom.dcmread(file_names[0])
voxel_size = (ds.PixelSpacing[0], ds.PixelSpacing[1], ds.SliceThickness)
voxel_size

('1.5625', '1.5625', '3.27')

In [15]:
# Ler máscaras e salvar como DICOM
path_list = glob.glob('G:/My Drive/Projetos Python/Image Processing/Hoffman_Templates/Mascaras/GM manipulada/*.tif')
masks_list = []
for path in path_list:
  name = path.split('/')[-1].split('.')[0].split('\\')[-1]
  masks = tiff.imread(path)
  resize_masks = cv.resize(masks, (256, 256))
  masks_list.append(resize_masks)
  create_DICOM(resize_masks, name, 'G:/My Drive/Projetos Python/Image Processing/Hoffman_Templates/Mascaras/GM resized/')

In [16]:
# Ler DRO e salvar como DICOM
path_list = glob.glob('G:/My Drive/Projetos Python/Image Processing/DATA SPECTRUM DIGITAL PHANTOM/JSNM mask/*.tif')
dro_list = []
for path in path_list:
  name = path.split('/')[-1].split('.')[0].split('\\')[-1]
  dro = tiff.imread(path)
  channels = cv.split(dro)
  dro = cv.merge([channels[0]]) 
  resize_dro = cv.resize(dro, (256, 256))
  dro_list.append(resize_dro)
  create_DICOM(resize_dro, name, 'G:/My Drive/Projetos Python/Image Processing/DATA SPECTRUM DIGITAL PHANTOM/resized DRO/')

# Visualização doas máscaras

In [17]:
fig = px.imshow(np.asarray(dro_list), animation_frame=0, binary_string=True, labels=dict(animation_frame="Slice"))
fig.show()